In [ ]:
# PRACTICAL 05: Seasonal Indices using Ratio-to-Trend Method
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Load dataset
path = "Symphony-Data.csv"  # Update with your actual path
df = pd.read_csv(path)

# Step 2: Preprocess
df = df.drop(columns=['OPEN', 'HIGH', 'LOW', 'VOLUME', 'CHANGE(%)'])
df['DATE'] = pd.to_datetime(df['DATE'], format="%d-%b-%y")
df.sort_values('DATE', inplace=True)

# Step 3: Extract year and month
df['Year'] = df['DATE'].dt.year
df['Month'] = df['DATE'].dt.month

# Step 4: Calculate monthly means for each year
monthly_avg = df.groupby(['Month', 'Year'])['PRICE'].mean().unstack()

# Step 5: Plot monthly averages
monthly_avg.plot(marker='o', figsize=(12, 6), title="Month-wise Average Stock Prices")
plt.xlabel("Month")
plt.ylabel("Average Price")
plt.grid(True)
plt.legend(title="Year")
plt.tight_layout()
plt.show()

# Step 6: Calculate yearly trend (average price)
yearly_trend = monthly_avg.mean()

# Step 7: Compute ratio of each month to yearly trend
ratios = monthly_avg.copy()
for year in yearly_trend.index:
    ratios[f'ratio_{year}'] = monthly_avg[year] / yearly_trend[year]

# Step 8: Compute normalized seasonal indices
ratio_cols = [col for col in ratios.columns if 'ratio_' in str(col)]
normalized_indices = ratios[ratio_cols].mean(axis=1)

# Step 9: Plot normalized seasonal indices
plt.figure(figsize=(10, 5))
plt.plot(normalized_indices.index, normalized_indices, marker='o')
plt.title("Normalized Seasonal Indices")
plt.xlabel("Month")
plt.ylabel("Index")
plt.grid(True)
plt.tight_layout()
plt.show()

# Step 10: Deseasonalized data calculation (for all years)
monthly_mean = df.groupby(['Year', 'Month'])['PRICE'].mean().unstack()
deseasonalized = pd.DataFrame()
for month in range(1, 13):
    deseasonalized[month] = monthly_mean[month] / normalized_indices[month]

# Step 11: Plot deseasonalized prices
plt.figure(figsize=(12, 6))
for year in deseasonalized.index:
    plt.plot(deseasonalized.columns, deseasonalized.loc[year], marker='o', label=str(year))

plt.title("Deseasonalized Monthly Prices")
plt.xlabel("Month")
plt.ylabel("Price (Deseasonalized)")
plt.legend(title="Year")
plt.grid(True)
plt.tight_layout()
plt.show()
